Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
import pandas as pd
import numpy as np
from string import punctuation
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [3]:
# import sys
# !{sys.executable} -m pip install pymorphy2

In [4]:
from pymorphy2 import MorphAnalyzer

In [5]:
# !{sys.executable} -m pip install stop_words

In [6]:
from stop_words import get_stop_words

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [8]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = re.sub('[^А-Я,^а-я]', ' ', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [9]:
data = pd.read_excel("отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [10]:
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3
data['target'] = data['target'].astype(int)
data = data.drop(['Date', 'Rating'], axis=1)
data.head()

,Content,target
0,It just works!,1
1,В целом удобноное приложение...из минусов хотя...,1
2,Отлично все,1
3,Стал зависать на 1% работы антивируса. Дальше ...,1
4,"Очень удобно, работает быстро.",1


In [11]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_val = pd.read_csv("./data/val.csv")

In [12]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [13]:
df_test.head()

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."
3,204153,RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4,204154,"RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во..."


In [14]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [15]:
df_train['text'] = df_train['text'].apply(preprocess_text)
df_train.head()

,id,text,class
0,0,уезжаааааааать тожена хотеть уезжать,0
1,1,ребята девчата кино любовь завтра вотэтолюбовь,1
2,2,ктоненавидеть пробка ретвит,0
3,3,хотеться котлета покиевск запретный плод,1
4,4,босапоп есбосан бояться мороз,1


In [16]:
df_test['text'] = df_test['text'].apply(preprocess_text)
df_test.head()

,id,text
0,204150,тектоника рельефсамый ужасный мир мучение
1,204151,ходить запускать шар желание насна получиться ...
2,204152,хотеть лето ради направить ноготь яркий лак
3,204153,блин шипперила флораблум музатекно стеллуна по...
4,204154,блинкаждый воскресение утром выходить какашкап...


In [17]:
df_val['text'] = df_val['text'].apply(preprocess_text)
df_val.head()

,id,text,class
0,181467,максимальный репост вромайдан,1
1,181468,зп ежегодно индексировать индекс инфляция тари...,0
2,181469,нехуй мнен,0
3,181470,нога ахахи когдаахах честнна помнить завтра шк...,0
4,181471,хотеть сделать получать,0


In [18]:
data['Content'] = data['Content'].apply(preprocess_text)
data.head()

,Content,target
0,,1
1,целое удобноной приложениеиз минус хотеть боль...,1
2,отлично,1
3,зависать работа антивирус ранее пользоваться н...,1
4,удобно работать быстро,1


In [19]:
train_corpus = " ".join(df_train["text"])

In [20]:
tokens = word_tokenize(train_corpus)

In [21]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [22]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [23]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [24]:
tokens_filtered_top[:10]

['хотеть',
 'знать',
 'ян',
 'мочь',
 'новый',
 'любить',
 'завтра',
 'вс',
 'мой',
 'хороший']

In [25]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [28]:
def text_to_sequence(text, maxlen, vocabulary=vocabulary):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [29]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [30]:
x_train.shape

(181467, 40)

In [31]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 161,
         7], dtype=int32)

1. Учим conv сеть для классификации - выбить auc выше 0.95

In [35]:
# import sys
# !{sys.executable} -m pip install keras

In [37]:
# !{sys.executable} -m pip install tensorflow

     |████████████████████████████████| 165.1MB 49kB/s  eta 0:00:01    |███████▋                        | 39.1MB 1.1MB/s eta 0:01:53     |████████████████████████████    | 144.6MB 90kB/s eta 0:03:47     |██████████████████████████████▎ | 156.0MB 857kB/s eta 0:00:11     |███████████████████████████████▏| 160.6MB 394kB/s eta 0:00:12
     |████████████████████████████████| 460kB 630kB/s eta 0:00:01
     |████████████████████████████████| 3.4MB 946kB/s eta 0:00:01
     |████████████████████████████████| 133kB 736kB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 984kB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 823kB/s eta 0:00:01
     |████████████████████████████████| 61kB 985kB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 824kB/s eta 0:00:01
     |████████████████████████████████| 71kB 956kB/s eta 0:00:01
     |████████████████████████████████| 51kB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 788kB 745kB/s eta 0:00:01
     |████

  Using cached https://files.pythonhosted.org/packages/62/1e/a94a8d635fa3ce4cfc7f506003548d0a2447ae76fd5ca53932970fe3053f/pyasn1-0.4.8-py2.py3-none-any.whl
     |████████████████████████████████| 153kB 953kB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4832 sha256=a04a0c2c188e9c6c0c4e3ba665398ca9f98a8083b8497eee8c5a12fec9047d27
  Stored in directory: /Users/levsolovev/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built termcolor
  Found existing installation: h5py 2.9.0
    Uninstalling h5py-2.9.0:
      Successfully uninstalled h5py-2.9.0


In [38]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [39]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [40]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len
                   ))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [41]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
  1/319 [..............................] - ETA: 0s - loss: 0.6926 - accuracy: 0.5488WARNING:tensorflow:From /Users/levsolovev/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
319/319 [==============================] - 30s 94ms/step - loss: 0.6539 - accuracy: 0.6010 - val_loss: 0.6437 - val_accuracy: 0.6113
Epoch 2/20
319/319 [==============================] - 29s 92ms/step - loss: 0.6445 - accuracy: 0.6129 - val_loss: 0.6435 - val_accuracy: 0.6110
Epoch 3/20
319/319 [==============================] - 29s 92ms/step - loss: 0.6410 - accuracy: 0.6157 - val_loss: 0.6430 - val_accuracy: 0.6106
Epoch 4/20
319/319 [==============================] - 29s 92ms/step - loss: 0.6375 - accuracy: 0.6192 - val_loss: 0.6425 - val_accuracy: 0.6123
Epoch 5/20
319/319 [=====================

In [43]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 24ms/step - loss: 0.6482 - accuracy: 0.6062


Test score: 0.6481879353523254
Test accuracy: 0.6061808466911316


2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [47]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook

In [52]:
modelW2V = Word2Vec(sentences=df_train['text'].apply(str.split), size=50, window=5, min_count=1)

In [48]:
vect_idf = TfidfVectorizer()
vect_idf.fit_transform(df_train['text'])
tfidf = dict(zip(vect_idf.get_feature_names(), vect_idf.idf_))

In [49]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(50)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            vector_w2v += modelW2V[wrd]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

def get_vect_idf(txt):
    vector_w2v = np.zeros(50)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            vector_w2v += modelW2V[wrd]*tfidf.get(wrd, 1.)
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [53]:
arr_vect = []
for txt in tqdm_notebook(df_train['text']):
    arr_vect.append(get_vect_mean(txt))
    
arr_vect_valid = []
for txt in tqdm_notebook(df_val['text']):
    arr_vect_valid.append(get_vect_mean(txt))
    
train_w2v = np.asarray(arr_vect)    
valid_w2v = np.asarray(arr_vect_valid)

In [54]:
lgr_w2v = LogisticRegression()

In [55]:
lgr_w2v.fit(train_w2v, df_train['class'].to_numpy())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred = lgr_w2v.predict(valid_w2v)

In [57]:
accuracy_score(df_val['class'].to_numpy(), y_pred)

0.620420579288454